In [13]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True) # limit gpu memory
    
from psychrnn.tasks.task import Task
from psychrnn.backend.models.basic import Basic
from psychrnn.backend.models.lstm import LSTM
import numpy as np
import scipy.io
import pandas as pd
import pickle
import numpy as np
from itertools import product
from typing import *
import os
import json
from uuid import uuid4, UUID
from pathlib import Path
from tqdm import tqdm

# Load teena's data that's organized-ish

In [2]:
teena_data = pd.read_pickle("./teena_df.pickle")

In [3]:
teena_data

,traj,file,sf
0,"[[222.1209111213684, 168.07275390625], [221.94...",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,True
1,"[[169.10160207748413, 175.90780782699585], [16...",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,False
2,"[[230.9175843000412, 184.91174030303955], [231...",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,False
3,"[[211.2449939250946, 173.9231698513031], [211....",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,True
4,"[[294.99583292007446, 73.52325487136841], [294...",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,True
...,...,...,...
924,"[[105.88987445831299, 211.26179087162018], [10...",/data2/kushal/inter_region/JoyM2/072619_PT/Beh...,True
925,"[[99.2321879863739, 205.92135858535767], [99.0...",/data2/kushal/inter_region/JoyM2/072619_PT/Beh...,True
926,"[[101.94186615943909, 202.87839889526367], [10...",/data2/kushal/inter_region/JoyM2/072619_PT/Beh...,True
927,"[[99.41120111942291, 205.7405904531479], [99.7...",/data2/kushal/inter_region/JoyM2/072619_PT/Beh...,True


# Use only success data

In [4]:
# use only success data
teena_data = teena_data[teena_data["sf"] == True]
teena_data

,traj,file,sf
0,"[[222.1209111213684, 168.07275390625], [221.94...",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,True
3,"[[211.2449939250946, 173.9231698513031], [211....",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,True
4,"[[294.99583292007446, 73.52325487136841], [294...",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,True
5,"[[285.5730181336403, 78.19226586818695], [285....",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,True
6,"[[279.1388177871704, 78.51209497451782], [279....",/data2/kushal/inter_region/JoyJ2/121819_IT/Beh...,True
...,...,...,...
924,"[[105.88987445831299, 211.26179087162018], [10...",/data2/kushal/inter_region/JoyM2/072619_PT/Beh...,True
925,"[[99.2321879863739, 205.92135858535767], [99.0...",/data2/kushal/inter_region/JoyM2/072619_PT/Beh...,True
926,"[[101.94186615943909, 202.87839889526367], [10...",/data2/kushal/inter_region/JoyM2/072619_PT/Beh...,True
927,"[[99.41120111942291, 205.7405904531479], [99.7...",/data2/kushal/inter_region/JoyM2/072619_PT/Beh...,True


# `dstack`, shape should be [n_frames_vid, xy, n_trials]

In [5]:
teena_data_array = np.dstack(teena_data["traj"])
teena_data_array.shape

(4997, 2, 702)

# Use only first 2500 frames, behavior usually occurs within this timespan

In [6]:
traj_data = teena_data_array[:2500, :, :100]

In [7]:
traj_data.shape

(2500, 2, 100)

# Define behavioral task, basically just samples `traj_data`

In [8]:
class Reach2Grab(Task):
    def __init__(self, dt, tau, T, N_batch):
        super(Reach2Grab, self).__init__(1, 2, dt, tau, T, N_batch)

    def generate_trial_params(self, batch, trial):
        """"Define parameters for each trial.

        Using a combination of randomness, presets, and task attributes, define the necessary trial parameters.

        Args:
            batch (int): The batch number that this trial is part of.
            trial (int): The trial number of the trial within the batch.

        Returns:
            dict: Dictionary of trial parameters.

        """

        # ----------------------------------
        # Define parameters of a trial
        # ----------------------------------
        params = dict()
        params["trial_ix"] = np.random.randint(0, traj_data.shape[2])

        return params

    def trial_function(self, time, params):
        """ Compute the trial properties at the given time.

        Based on the params compute the trial stimulus (x_t), correct output (y_t), and mask (mask_t) at the given time.

        Args:
            time (int): The time within the trial (0 <= time < T).
            params (dict): The trial params produced generate_trial_params()

        Returns:
            tuple:

            x_t (ndarray(dtype=float, shape=(N_in,))): Trial input at time given params.
            y_t (ndarray(dtype=float, shape=(N_out,))): Correct trial output at time given params.
            mask_t (ndarray(dtype=bool, shape=(N_out,))): True if the network should train to match the y_t, False if the network should ignore y_t when training.

        """
        x_t = 1 # just a "go" cue
        y_t = traj_data[time, :, params["trial_ix"]]
        
        mask_t = np.ones(shape=y_t.shape, dtype=bool)
        
        # trajectory that matches direction 
        
        # y_t returns a trajectory

        return x_t, y_t, mask_t

# functions to save and load models to a dir because PsychRNN doesn't implement this properly ¯\\\_(ツ)_/¯

In [9]:
def save_model(m: Union[Basic, LSTM], u: Union[str, UUID]):
    d = Path(f"/home/kushalk/repos/rnn-go-brr/rnngobrr/models/{u}")
    weights_path = d.joinpath("weights")
    params_path = d.joinpath("params.json")
    os.mkdir(d)
    
    model.save(weights_path)
    json.dump(model.params, open(params_path, "w"))


def load_model(m: str, u: Union[str, UUID]) -> Union[Basic, LSTM]:
    d = Path(f"/home/kushalk/repos/rnn-go-brr/rnngobrr/models/{u}")
    # need to specify extension, these guys are so inconsistent (╯°□°)╯︵ ┻━┻
    weights_path = d.joinpath("weights.npz")
    params_path = d.joinpath("params.json")
    
    params = json.load(open(params_path, "r"))
    params["name"] = str(u)
    params["load_weights_path"] = weights_path
    
    if m == "Basic":
        return Basic(params)
    elif m == "LSTM":
        return LSTM(params)

# `gs_params` defines search range for each param

In [10]:
gs_params = {
    "dt": list(range(1, 13, 2)),
    "tau": list(range(10, 201, 50)),
    "T": [traj_data.shape[0]], # num frames to use
    "learning_rate": 1 / 10 ** np.arange(4, 10, 2),
    "training_iters": list(range(100, 501, 100)),
    "N_rec": list(range(100, 501, 200)), # number of neurons
}

# this is the same order that must be used when using the iterator!!
# dict keys don't return stuff in order, so gs_params.keys() shouldn't be used for this!!
gs_param_names = [
    "dt",
    "tau",
    "T",
    "learning_rate",
    "training_iters",
    "N_rec"
]

gs_params

{'dt': [1, 3, 5, 7, 9, 11],
 'tau': [10, 60, 110, 160],
 'T': [2500],
 'learning_rate': array([1.e-04, 1.e-06, 1.e-08]),
 'training_iters': [100, 200, 300, 400, 500],
 'N_rec': [100, 300, 500]}

# Create iterator using all possible combinations of search parameters, check number of variants

In [20]:
iterator = product(*[gs_params[k] for k in gs_param_names])
n_combos = len(list(iterator))
print(n_combos)

# need to make it again because calling `list()` destroys it
iterator = product(*[gs_params[k] for k in gs_param_names])

1080


In [ ]:
columns = gs_param_names + ["uuid", "arch"]

df = pd.DataFrame(columns=columns)

for dt, tau, T, learning_rate, training_iters, N_rec in tqdm(iterator, total=n_combos):
    u = uuid4() # to keep track of this model
    
    task = Reach2Grab(dt = dt, tau = tau, T = T, N_batch = 128)
    
    network_params = task.get_task_params()
    network_params['name'] = str(u)
    network_params['N_rec'] = N_rec
    
    model_iter = Basic(network_params)

    train_params = {
        "training_iters": training_iters,
        "learning_rate": learning_rate,
    }
    
    model_iter.train(task, train_params)
    save_model(model_iter, u)
    
    # create df row
    s = pd.Series(
        {
            "uuid": str(u),
            "arch": "Basic",
            "dt": dt,
            "tau": tau,
            "T": T,
            "learning_rate": learning_rate,
            "training_iters": training_iters,
            "N_rec": N_rec
        }
    )

    
    # append to dataframe
    df.loc[df.index.size] = s
    
    df.to_pickle("./models_dataframe.pickle")
    model_iter.destruct()
    # ---------------------- Test the trained model ---------------------------
    # x,target_output,mask, trial_params = task.get_trial_batch() # get pd task inputs and outputs
    # model_output, model_state = model.test(x) # run the model on input x

  0%|                                                                                | 0/1080 [00:00<?, ?it/s]2022-11-15 23:31:26.824227: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 23:31:26.825866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 23:31:26.826433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 23:31:26.826924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node

In [70]:
[gridsearch_params[k] for k in list(gridsearch_params.keys())]

[[1, 3, 5, 7, 9, 11],
 [10, 60, 110, 160],
 [2000],
 array([1.e-04, 1.e-06, 1.e-08]),
 [100, 200, 300, 400, 500]]